<a href="https://colab.research.google.com/github/DaniloRando/BlueLumen/blob/main/Finanza_Agevolata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Installa dipendenze
!pip install selenium beautifulsoup4 requests

# Assicurati di avere ChromeDriver installato
# Poi esegui solo il BLOCCO 2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.7/512.7 kB 51.6 MB/s eta 0:00:00


In [ ]:
# ============================================================
# SETUP SELENIUM - Esegui questo blocco PRIMA di tutto
# ============================================================

import os
import sys

def setup_selenium():
    """Setup Selenium per Google Colab"""
    print("🔧 SETUP SELENIUM PER GOOGLE COLAB")
    print("="*60)

    try:
        import google.colab
        IN_COLAB = True
        print("✅ Ambiente: Google Colab")
    except:
        IN_COLAB = False
        print("ℹ️ Ambiente: Locale")

    if IN_COLAB:
        print("\n📦 Installazione dipendenze...")

        # Aggiorna sistema
        print("  → Aggiornamento sistema...")
        os.system('apt-get update > /dev/null 2>&1')

        # Installa Chrome e ChromeDriver
        print("  → Installazione ChromeDriver...")
        os.system('apt install -y chromium-chromedriver > /dev/null 2>&1')

        # Installa librerie Python
        print("  → Installazione librerie Python...")
        os.system('pip install -q selenium beautifulsoup4 requests')

        print("\n✅ Setup completato!")

    else:
        print("\n⚠️ Non sei in Colab.")
        print("   Assicurati di avere installato:")
        print("   pip install selenium beautifulsoup4 requests")
        print("   E di avere ChromeDriver nel PATH")

    print("="*60)

    # Verifica installazione
    try:
        from selenium import webdriver
        from selenium.webdriver.chrome.options import Options
        print("\n✅ Selenium importato correttamente")
        return True
    except ImportError as e:
        print(f"\n❌ Errore import Selenium: {e}")
        return False

# ESEGUI SETUP
setup_selenium()

🔧 SETUP SELENIUM PER GOOGLE COLAB
✅ Ambiente: Google Colab

📦 Installazione dipendenze...
  → Aggiornamento sistema...
  → Installazione ChromeDriver...
  → Installazione librerie Python...

✅ Setup completato!

✅ Selenium importato correttamente


True

In [ ]:
import os
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import time
from pathlib import Path
import csv
from datetime import datetime

# Import Selenium
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException


class PDFDownloader:
    def __init__(self, base_download_path='/content/bandi_pdf'):
        self.base_download_path = base_download_path

        # Session HTTP per richieste normali (come sempre)
        self.session = requests.Session()
        self.session.headers.update({
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        })

        # Selenium driver (nuovo - inizializzato solo quando serve)
        self.driver = None

        # Metadati (come sempre)
        self.metadata = []

    def sanitize_filename(self, filename):
        """Rimuove caratteri non validi dal nome del file"""
        invalid_chars = '<>:"/\\|?*'
        for char in invalid_chars:
            filename = filename.replace(char, '_')
        return filename[:200]

    def add_metadata(self, fonte, nome_bando, link_bando, file_scaricato, tipo_file, data_download):
        """Aggiunge metadati al registro"""
        self.metadata.append({
            'fonte': fonte,
            'nome_bando': nome_bando,
            'link_bando': link_bando,
            'file_scaricato': file_scaricato,
            'tipo_file': tipo_file,
            'data_download': data_download,
            'percorso_completo': os.path.join(self.base_download_path,
                                             self.sanitize_filename(f"{fonte}_{nome_bando}"),
                                             file_scaricato)
        })

    def save_metadata_csv(self):
        """Salva tutti i metadati in un file CSV"""
        csv_path = os.path.join(self.base_download_path, 'bandi_metadata.csv')

        if not self.metadata:
            print("\nNessun metadato da salvare")
            return

        try:
            with open(csv_path, 'w', newline='', encoding='utf-8-sig') as csvfile:
                fieldnames = ['fonte', 'nome_bando', 'link_bando', 'file_scaricato',
                            'tipo_file', 'data_download', 'percorso_completo']
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

                writer.writeheader()
                for row in self.metadata:
                    writer.writerow(row)

            print(f"\n✓ Metadati salvati in: {csv_path}")
            print(f"  Totale record: {len(self.metadata)}")

        except Exception as e:
            print(f"\n✗ Errore nel salvare il CSV: {str(e)}")

    def download_file(self, url, save_path, fonte, nome_bando, link_bando):
        """Scarica un file e lo salva registrando i metadati"""
        try:
            response = self.session.get(url, timeout=30, stream=True)
            response.raise_for_status()

            with open(save_path, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    f.write(chunk)

            file_extension = os.path.splitext(save_path)[1].lower()
            tipo_file = file_extension[1:] if file_extension else 'unknown'

            self.add_metadata(
                fonte=fonte,
                nome_bando=nome_bando,
                link_bando=link_bando,
                file_scaricato=os.path.basename(save_path),
                tipo_file=tipo_file,
                data_download=datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            )

            print(f"✓ Scaricato: {os.path.basename(save_path)}")
            return True

        except Exception as e:
            print(f"✗ Errore download {url}: {str(e)}")
            return False

    def save_bando_info(self, folder_path, fonte, nome_bando, link_bando):
        """Salva un file info.txt con le informazioni del bando"""
        info_path = os.path.join(folder_path, '_info_bando.txt')
        try:
            with open(info_path, 'w', encoding='utf-8') as f:
                f.write(f"FONTE: {fonte}\n")
                f.write(f"NOME BANDO: {nome_bando}\n")
                f.write(f"LINK: {link_bando}\n")
                f.write(f"DATA DOWNLOAD: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        except Exception as e:
            print(f"  Avviso: impossibile salvare info.txt: {str(e)}")

    # =====================================================
    # METODI SELENIUM (NUOVI)
    # =====================================================

    def init_selenium(self):
        """Inizializza Selenium WebDriver"""
        if self.driver is not None:
            return True

        print("\n🚀 Inizializzazione Selenium WebDriver...")

        try:
            chrome_options = Options()
            chrome_options.add_argument('--headless')
            chrome_options.add_argument('--no-sandbox')
            chrome_options.add_argument('--disable-dev-shm-usage')
            chrome_options.add_argument('--disable-gpu')
            chrome_options.add_argument('--window-size=1920,1080')
            chrome_options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36')

            self.driver = webdriver.Chrome(options=chrome_options)
            self.driver.set_page_load_timeout(30)

            print("✅ Selenium inizializzato")
            return True

        except Exception as e:
            print(f"❌ Errore inizializzazione Selenium: {e}")
            self.driver = None
            return False

    def close_selenium(self):
        """Chiude Selenium WebDriver"""
        if self.driver:
            try:
                self.driver.quit()
                print("✅ Selenium chiuso")
            except:
                pass
            self.driver = None

    # =====================================================
    # MIMIT - COME PRIMA (usa solo requests)
    # =====================================================

    def process_mimit(self):
        """Processa il sito MIMIT con paginazione"""
        print("\n" + "="*60)
        print("FONTE 1: MIMIT - https://www.mimit.gov.it/it/incentivi")
        print("="*60)

        fonte = "MIMIT"
        base_url = "https://www.mimit.gov.it"

        # FASE 1: RACCOLTA
        bandi_dict = {}
        start = 0
        page_num = 1

        print("\n🔍 FASE 1: RACCOLTA BANDI DA TUTTE LE PAGINE")

        while True:
            main_page = f"{base_url}/it/incentivi?start={start}"
            print(f"  📄 Scansione pagina {page_num} (start={start})...", end=" ")

            try:
                response = self.session.get(main_page, timeout=30)
                response.raise_for_status()
                soup = BeautifulSoup(response.content, 'html.parser')

                page_bandi_count = 0
                for row in soup.find_all('div', class_='row m-0'):
                    h2 = row.find('h2')
                    if h2 and h2.find('a'):
                        link = h2.find('a')['href']
                        title = h2.find('a').get_text(strip=True)
                        full_link = urljoin(base_url, link)

                        if full_link not in bandi_dict:
                            bandi_dict[full_link] = title
                            page_bandi_count += 1

                if page_bandi_count == 0:
                    print(f"✓ Nessun nuovo bando. Fine.")
                    break

                print(f"✓ {page_bandi_count} nuovi bandi (totale: {len(bandi_dict)})")

                next_button = soup.find('a', class_='page-link', attrs={'aria-label': 'Vai alla pagina avanti'})

                if not next_button:
                    print(f"  ℹ️ Ultima pagina raggiunta.")
                    break

                start += 20
                page_num += 1
                time.sleep(2)

            except Exception as e:
                print(f"✗ Errore: {str(e)}")
                break

        print(f"\n✅ TOTALE BANDI UNICI TROVATI: {len(bandi_dict)} bandi")

        # FASE 2: DOWNLOAD
        print(f"\n📥 FASE 2: DOWNLOAD DOCUMENTI DA {len(bandi_dict)} BANDI\n")

        for idx, (url, title) in enumerate(bandi_dict.items(), 1):
            print(f"[{idx}/{len(bandi_dict)}] {title}")

            folder_name = self.sanitize_filename(f"{fonte}_{title}")
            folder_path = os.path.join(self.base_download_path, folder_name)
            os.makedirs(folder_path, exist_ok=True)

            self.save_bando_info(folder_path, fonte, title, url)
            self.extract_pdfs_from_mimit_page(url, folder_path, base_url, fonte, title, url)
            time.sleep(2)

    def extract_pdfs_from_mimit_page(self, page_url, folder_path, base_url, fonte, nome_bando, link_bando):
        """Estrae PDF da una pagina MIMIT cercando nella sezione Normativa"""
        try:
            response = self.session.get(page_url, timeout=30)
            response.raise_for_status()
            soup = BeautifulSoup(response.content, 'html.parser')

            normativa_section = None
            for h2 in soup.find_all('h2'):
                if 'normativa' in h2.get_text().lower():
                    normativa_section = h2
                    break

            if not normativa_section:
                print("  ⚠️ Sezione Normativa non trovata")
                return

            current = normativa_section.find_next()
            pdf_count = 0

            while current and current.name != 'h2':
                if current.name == 'a' and current.get('href'):
                    href = current['href']
                    full_url = urljoin(base_url, href)

                    if href.lower().endswith('.pdf'):
                        filename = os.path.basename(urlparse(href).path)
                        filename = self.sanitize_filename(filename)
                        save_path = os.path.join(folder_path, filename)

                        if self.download_file(full_url, save_path, fonte, nome_bando, link_bando):
                            pdf_count += 1

                    elif any(href.lower().endswith(ext) for ext in ['.docx', '.doc', '.xlsx', '.xls']):
                        filename = os.path.basename(urlparse(href).path)
                        filename = self.sanitize_filename(filename)
                        save_path = os.path.join(folder_path, filename)

                        if self.download_file(full_url, save_path, fonte, nome_bando, link_bando):
                            pdf_count += 1

                    elif '/it/' in href and not href.lower().endswith(('.jpg', '.jpeg', '.png', '.gif')):
                        self.check_page_for_documents(full_url, folder_path, base_url, fonte, nome_bando, link_bando)

                current = current.find_next()

            print(f"  → {pdf_count} documenti scaricati")

        except Exception as e:
            print(f"  ✗ Errore: {str(e)}")

    def check_page_for_documents(self, page_url, folder_path, base_url, fonte, nome_bando, link_bando):
        """Controlla una pagina per eventuali documenti scaricabili"""
        try:
            response = self.session.get(page_url, timeout=30)
            response.raise_for_status()
            soup = BeautifulSoup(response.content, 'html.parser')

            for link in soup.find_all('a', href=True):
                href = link['href']
                if any(href.lower().endswith(ext) for ext in ['.pdf', '.docx', '.doc', '.xlsx', '.xls']):
                    full_url = urljoin(base_url, href)
                    filename = os.path.basename(urlparse(href).path)
                    filename = self.sanitize_filename(filename)
                    save_path = os.path.join(folder_path, filename)

                    self.download_file(full_url, save_path, fonte, nome_bando, link_bando)

        except Exception as e:
            pass

    # =====================================================
    # REGIONE LOMBARDIA - NUOVO (usa Selenium)
    # =====================================================

    def process_regione_lombardia(self):
        """Processa il sito Regione Lombardia con paginazione Selenium"""
        print("\n" + "="*60)
        print("FONTE 2: Regione Lombardia (Selenium)")
        print("="*60)

        # Inizializza Selenium
        if not self.init_selenium():
            print("❌ Impossibile processare Regione Lombardia senza Selenium")
            return

        fonte = "Regione Lombardia"
        base_url = "https://www.bandi.regione.lombardia.it"
        page_url = f"{base_url}/servizi/servizio/bandi/sostegno-attivita-imprenditoriali"

        # FASE 1: RACCOLTA con Selenium
        bandi_dict = {}
        current_page = 1
        max_pages = None

        print("\n🔍 FASE 1: RACCOLTA BANDI CON SELENIUM")

        try:
            print(f"\n📄 Carico pagina iniziale...")
            self.driver.get(page_url)
            time.sleep(3)  # Attendi caricamento iniziale

            # Scopri numero totale pagine
            try:
                pagination = self.driver.find_element(By.CSS_SELECTOR, "a.page-link.page-num[data-max-page-num]")
                max_pages = int(pagination.get_attribute('data-max-page-num'))
                print(f"✅ Trovate {max_pages} pagine totali")
            except:
                max_pages = 37  # Fallback
                print(f"⚠️ Numero pagine non rilevato, uso default: {max_pages}")

            while current_page <= max_pages:
                print(f"  📄 Scansione pagina {current_page} di {max_pages}...", end=" ")

                # Attendi che le card siano visibili
                try:
                    WebDriverWait(self.driver, 10).until(
                        EC.presence_of_element_located((By.CLASS_NAME, "card-body"))
                    )
                except TimeoutException:
                    print(f"⚠️ Timeout")
                    break

                # Estrai HTML
                page_source = self.driver.page_source
                soup = BeautifulSoup(page_source, 'html.parser')

                # Estrai bandi (IDENTICO ALLA VERSIONE PRECEDENTE)
                page_bandi_count = 0
                for card_body in soup.find_all('div', class_='card-body'):

                    scopri_link = None
                    card_footer = card_body.find('div', class_='it-card-footer')

                    if card_footer:
                        for link in card_footer.find_all('a', href=True):
                            if '/dettaglio/' in link.get('href', '') and 'btn-primary' in link.get('class', []):
                                scopri_link = link
                                break

                    if not scopri_link:
                        for link in card_body.find_all('a', href=True):
                            if '/dettaglio/' in link.get('href', ''):
                                scopri_link = link
                                break

                    if scopri_link:
                        href = scopri_link['href']
                        full_link = urljoin(base_url, href)

                        if full_link not in bandi_dict:
                            title_elem = card_body.find('h4', class_='card-title')
                            if title_elem:
                                title = title_elem.get_text(strip=True).replace('...', '').strip()
                            else:
                                title = "Bando_senza_titolo"

                            code_elem = card_body.find('strong', class_='code')
                            code = code_elem.get_text(strip=True) if code_elem else ""

                            full_title = f"{code}_{title}" if code else title

                            bandi_dict[full_link] = full_title
                            page_bandi_count += 1

                print(f"✓ {page_bandi_count} nuovi bandi (totale: {len(bandi_dict)})")

                # Vai alla pagina successiva
                if current_page < max_pages:
                    try:
                        # Cerca e clicca pulsante "Avanti"
                        next_button = self.driver.find_element(By.CSS_SELECTOR, "a.page-link.page-up")

                        # Verifica se disabilitato
                        parent_li = next_button.find_element(By.XPATH, "..")
                        if 'disabled' in parent_li.get_attribute('class'):
                            print(f"  ℹ️ Fine paginazione (pulsante disabilitato)")
                            break

                        # Click con JavaScript (più affidabile)
                        self.driver.execute_script("arguments[0].click();", next_button)
                        time.sleep(3)  # Attendi caricamento nuova pagina

                    except NoSuchElementException:
                        print(f"  ℹ️ Pulsante 'Avanti' non trovato")
                        break
                    except Exception as e:
                        print(f"  ⚠️ Errore navigazione: {e}")
                        break

                current_page += 1

        except Exception as e:
            print(f"\n✗ Errore durante raccolta: {e}")
            import traceback
            traceback.print_exc()

        print(f"\n✅ TOTALE BANDI UNICI TROVATI: {len(bandi_dict)} bandi")

        if not bandi_dict:
            print("⚠️ Nessun bando trovato!")
            return

        # FASE 2: DOWNLOAD (usa requests, IDENTICO ALLA VERSIONE PRECEDENTE)
        print(f"\n📥 FASE 2: DOWNLOAD DOCUMENTI DA {len(bandi_dict)} BANDI\n")

        for idx, (url, title) in enumerate(bandi_dict.items(), 1):
            print(f"[{idx}/{len(bandi_dict)}] {title}")

            folder_name = self.sanitize_filename(f"{fonte}_{title}")
            folder_path = os.path.join(self.base_download_path, folder_name)
            os.makedirs(folder_path, exist_ok=True)

            self.save_bando_info(folder_path, fonte, title, url)
            self.extract_pdfs_from_lombardia_page(url, folder_path, base_url, fonte, title, url)
            time.sleep(2)

    def extract_pdfs_from_lombardia_page(self, page_url, folder_path, base_url, fonte, nome_bando, link_bando):
        """Estrae PDF da una pagina Regione Lombardia"""
        try:
            response = self.session.get(page_url, timeout=30)
            response.raise_for_status()
            soup = BeautifulSoup(response.content, 'html.parser')

            pdf_count = 0
            downloaded_files = set()

            for right_zone in soup.find_all('div', class_='it-right-zone'):
                parent_link = right_zone.find_parent('a')
                if parent_link and parent_link.get('href'):
                    href = parent_link['href']
                    full_url = urljoin(base_url, href)

                    text_elem = right_zone.find('span', class_='text')
                    if text_elem:
                        text = text_elem.get_text(strip=True)
                        filename = self.sanitize_filename(text)

                        if not any(filename.lower().endswith(ext) for ext in ['.pdf', '.docx', '.doc', '.xlsx', '.xls']):
                            url_ext = os.path.splitext(urlparse(full_url).path)[1]
                            if url_ext:
                                filename += url_ext
                            else:
                                filename += '.pdf'

                        save_path = os.path.join(folder_path, filename)

                        if filename not in downloaded_files:
                            if self.download_file(full_url, save_path, fonte, nome_bando, link_bando):
                                pdf_count += 1
                                downloaded_files.add(filename)

            for link in soup.find_all('a', href=True):
                href = link['href']
                if any(href.lower().endswith(ext) for ext in ['.pdf', '.docx', '.doc', '.xlsx', '.xls']):
                    full_url = urljoin(base_url, href)
                    filename = os.path.basename(urlparse(href).path)
                    filename = self.sanitize_filename(filename)
                    save_path = os.path.join(folder_path, filename)

                    if filename not in downloaded_files and not os.path.exists(save_path):
                        if self.download_file(full_url, save_path, fonte, nome_bando, link_bando):
                            pdf_count += 1
                            downloaded_files.add(filename)

            print(f"  → {pdf_count} documenti scaricati")

        except Exception as e:
            print(f"  ✗ Errore: {str(e)}")

    # =====================================================
    # REPORTING (IDENTICO ALLE VERSIONI PRECEDENTI)
    # =====================================================

    def generate_summary_report(self):
        """Genera un report riassuntivo delle operazioni"""
        print("\n" + "="*60)
        print("REPORT FINALE")
        print("="*60)

        if not self.metadata:
            print("\nNessun file scaricato")
            return

        fonti = {}
        tipi_file = {}
        bandi = set()

        for item in self.metadata:
            fonte = item['fonte']
            tipo = item['tipo_file']
            bando = item['nome_bando']

            fonti[fonte] = fonti.get(fonte, 0) + 1
            tipi_file[tipo] = tipi_file.get(tipo, 0) + 1
            bandi.add(f"{fonte}::{bando}")

        print(f"\n📊 STATISTICHE:")
        print(f"  • Totale file scaricati: {len(self.metadata)}")
        print(f"  • Totale bandi processati: {len(bandi)}")

        print(f"\n📁 PER FONTE:")
        for fonte, count in fonti.items():
            print(f"  • {fonte}: {count} file")

        print(f"\n📄 PER TIPO FILE:")
        for tipo, count in sorted(tipi_file.items()):
            print(f"  • .{tipo}: {count} file")

        print(f"\n💾 File salvati in: {self.base_download_path}")

    # =====================================================
    # RUNNER PRINCIPALE
    # =====================================================

    def run(self):
        """Esegue il download da tutte le fonti"""
        print("\n" + "="*60)
        print("AVVIO SCARICAMENTO PDF DA BANDI")
        print("="*60)

        os.makedirs(self.base_download_path, exist_ok=True)
        print(f"\nCartella download: {self.base_download_path}\n")

        try:
            # Processa MIMIT (requests)
            self.process_mimit()

            # Processa Regione Lombardia (Selenium)
            self.process_regione_lombardia()

        finally:
            # Chiudi Selenium
            self.close_selenium()

        # Salva CSV
        self.save_metadata_csv()

        # Report
        self.generate_summary_report()

        print("\n" + "="*60)
        print("COMPLETATO!")
        print("="*60)


# =====================================================
# ESECUZIONE
# =====================================================

if __name__ == "__main__":
    downloader = PDFDownloader()
    downloader.run()


AVVIO SCARICAMENTO PDF DA BANDI

Cartella download: /content/bandi_pdf


FONTE 1: MIMIT - https://www.mimit.gov.it/it/incentivi

🔍 FASE 1: RACCOLTA BANDI DA TUTTE LE PAGINE
  📄 Scansione pagina 1 (start=0)... ✓ 20 nuovi bandi (totale: 20)
  📄 Scansione pagina 2 (start=20)... ✓ 20 nuovi bandi (totale: 40)
  📄 Scansione pagina 3 (start=40)... ✓ 20 nuovi bandi (totale: 60)
  📄 Scansione pagina 4 (start=60)... ✓ 20 nuovi bandi (totale: 80)
  📄 Scansione pagina 5 (start=80)... ✓ 20 nuovi bandi (totale: 100)
  📄 Scansione pagina 6 (start=100)... ✓ 20 nuovi bandi (totale: 120)
  📄 Scansione pagina 7 (start=120)... ✓ 20 nuovi bandi (totale: 140)
  📄 Scansione pagina 8 (start=140)... ✓ 20 nuovi bandi (totale: 160)
  📄 Scansione pagina 9 (start=160)... ✓ 20 nuovi bandi (totale: 180)
  📄 Scansione pagina 10 (start=180)... ✓ 20 nuovi bandi (totale: 200)
  📄 Scansione pagina 11 (start=200)... ✓ 20 nuovi bandi (totale: 220)
  📄 Scansione pagina 12 (start=220)... ✓ 20 nuovi bandi (totale: 240)
  📄 S

In [ ]:
# ============================================================
# UPLOAD SU GOOGLE DRIVE - METODO ZIP SEMPLICE
# ============================================================

import shutil
import os
from datetime import datetime
from google.colab import drive

def upload_to_drive_zip(source_folder='/content/bandi_pdf'):
    """
    Crea un ZIP dei bandi e lo carica su Google Drive
    Include timestamp nel nome file (formato: YYMMDD_bandi.zip)
    """

    print("\n📦 UPLOAD SU GOOGLE DRIVE (ZIP)")
    print("="*60)

    # 1. Verifica cartella sorgente
    if not os.path.exists(source_folder):
        print(f"❌ Cartella '{source_folder}' non trovata!")
        return

    # Conta file
    total_files = sum([len(files) for _, _, files in os.walk(source_folder)])
    print(f"\n📊 File da archiviare: {total_files}")

    # 2. Crea ZIP con timestamp
    timestamp = datetime.now().strftime('%y%m%d')  # Formato: YYMMDD
    zip_name = f'{timestamp}_bandi'
    zip_path = f'/content/{zip_name}'

    print(f"\n📦 Creazione archivio: {zip_name}.zip")
    print("   Attendere...")

    shutil.make_archive(zip_path, 'zip', source_folder)
    zip_file = f"{zip_path}.zip"

    # Dimensione ZIP
    size_mb = os.path.getsize(zip_file) / 1024 / 1024
    print(f"✅ ZIP creato: {size_mb:.2f} MB")

    # 3. Mount Google Drive
    print(f"\n📂 Mounting Google Drive...")
    drive.mount('/content/drive', force_remount=False)
    print("✅ Drive montato")

    # 4. Copia su Drive
    destination = f'/content/drive/MyDrive/Blue Lumen/Bandi/{zip_name}.zip'

    print(f"\n📤 Upload su Drive...")
    shutil.copy(zip_file, destination)

    print(f"\n✅ UPLOAD COMPLETATO!")
    print(f"📂 Percorso: Il mio Drive/{zip_name}.zip")
    print(f"📏 Dimensione: {size_mb:.2f} MB")
    print(f"💡 Per estrarre: apri Drive → click destro sul file → Estrai")
    print("="*60)


# =====================================================
# ESECUZIONE
# =====================================================

# Upload automatico dopo download
upload_to_drive_zip()


📦 UPLOAD SU GOOGLE DRIVE (ZIP)

📊 File da archiviare: 3974

📦 Creazione archivio: 251008_bandi.zip
   Attendere...
✅ ZIP creato: 1475.62 MB

📂 Mounting Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Drive montato

📤 Upload su Drive...

✅ UPLOAD COMPLETATO!
📂 Percorso: Il mio Drive/251008_bandi.zip
📏 Dimensione: 1475.62 MB
💡 Per estrarre: apri Drive → click destro sul file → Estrai
